In [49]:
# imports

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.plotly as py
%matplotlib inline
fig_size = [12,9]
plt.rcParams["figure.figsize"] = fig_size

plt.style.use("ggplot")

airports = pd.read_csv("../data/flight-delays/airports.csv")
airlines = pd.read_csv("../data/flight-delays/airlines.csv")
flights = pd.read_csv("../data/flight-delays/flights.csv")

In [2]:
airports = pd.read_csv("../data/flight-delays/airports.csv")
airlines = pd.read_csv("../data/flight-delays/airlines.csv")
flights = pd.read_csv("../data/flight-delays/flights.csv")

/Users/jakubkasan/coding/data-science/flight-delays/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
print("Airports: ",airports.shape)
print("Airlines: ",airlines.shape)
print("Flights: ",flights.shape)

Airports:  (322, 9)
Airlines:  (14, 2)
Flights:  (5819079, 31)


In [47]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
plt.scatter("LONGITUDE","LATITUDE",data=airports)
plt.show()

In [ ]:
flights["DESTINATION_AIRPORT"].value_counts()["ATL"]

In [4]:
arrival_counts = flights["DESTINATION_AIRPORT"].value_counts()
departure_counts = flights["ORIGIN_AIRPORT"].value_counts()
airports["TOTAL_ARRIVALS"] = [arrival_counts[apt] for apt in airports["IATA_CODE"]]
airports["TOTAL_DEPARTURES"] = [departure_counts[apt] for apt in airports["IATA_CODE"]]

In [5]:
total_departures = sum(departure_counts)
total_arrivals = sum(arrival_counts)

In [ ]:
s = [(x/total_departures)*20000 for x in airports["TOTAL_ARRIVALS"]]
plt.scatter("LONGITUDE","LATITUDE",data=airports,s=s,cmap="RdYlGn",c=s,alpha=0.75)
plt.show()

In [ ]:
flights.columns

In [19]:
from matplotlib import lines as mlines
import time

In [6]:
def newline(x1,y1,x2,y2):
    ax = plt.gca()
    line = mlines.Line2D([x1,x2],[y1,y2])
    ax.add_line(line)
    return line

In [39]:
def getRouteArgs(flights,airports):
    route_args = []
    unique_airports = airports["IATA_CODE"].unique()
    departures = [x for x in flights["ORIGIN_AIRPORT"].unique() if x in unique_airports]
    arrivals = [x for x in flights["DESTINATION_AIRPORT"].unique() if x in unique_airports]
    for departure in departures:
        for arrival in arrivals:
            route_args.append((departure,arrival))
    return(route_args)

def getRoutes(flights,airports,route_args,threads=None):
    start_time = time.time()
    if threads==None:
        results = list(map(lambda x: getRouteDate(flights,airports,x),route_args))
    else:
        def func(arg):
            getRouteDate(flights,airports,arg)
        results = threads.map(func,route_args)
    end_time = time.time()
    print(end_time-start_time)
    return results

def getRouteDate(flights,airports,routes_tup):
    departure = routes_tup[0]
    arrival = routes_tup[1]
    # get the index of relevant flights at start to save looking it up repeatedly
    selection = flights.loc[(flights["ORIGIN_AIRPORT"]==departure)&(flights["DESTINATION_AIRPORT"]==arrival),:]
    if selection.empty:
        return
    departure_loc = airports.loc[airports["IATA_CODE"]==departure,["LATITUDE","LONGITUDE"]]
    arrival_loc = airports.loc[airports["IATA_CODE"]==arrival,["LATITUDE","LONGITUDE"]]
    temp = dict(
        origin = departure,
        origin_lat = departure_loc["LATITUDE"],
        origin_long = departure_loc["LONGITUDE"],
        destination = arrival,
        destination_lat = arrival_loc["LATITUDE"],
        destination_long = arrival_loc["LONGITUDE"],
        num_flights = selection.shape[0],
        average_landing_delay = np.mean(selection["ARRIVAL_DELAY"])   
    )
    print("Finished {} to {}".format(departure,arrival))
    return(temp)
        

In [35]:
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool as RealPool

threads = RealPool(8)

#threads = ThreadPool(8)

In [46]:
#route_args[:10]
flights.loc[flights["DESTINATION_AIRPORT"]==route_args[0][1],:].values

array([[2015, 1, 1, ..., nan, nan, nan],
       [2015, 1, 1, ..., nan, nan, nan],
       [2015, 1, 1, ..., nan, nan, nan],
       ...,
       [2015, 12, 31, ..., nan, nan, nan],
       [2015, 12, 31, ..., nan, nan, nan],
       [2015, 12, 31, ..., nan, nan, nan]], dtype=object)

In [14]:
route_args = getRouteArgs(flights,airports)

In [41]:
myroutes = getRoutes(flights,airports,route_args[:100],threads)

Finished ANC to ORD
Finished ANC to SEA
Finished ANC to IAH
Finished ANC to LAX
Finished ANC to HNL
Finished ANC to SFO
Finished ANC to PDX
Finished ANC to DENFinished ANC to MSP

Finished ANC to SLCFinished ANC to DFW

Finished ANC to PHXFinished ANC to KOA

Finished ANC to ATL
Finished ANC to EWR
Finished ANC to OGG
Finished ANC to LGB
Finished ANC to OTZ
Finished ANC to LAS
Finished ANC to SCC
Finished ANC to JNU
104.49637794494629


In [42]:
myroutes

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [23]:
print(time.time()-time.time())

-9.5367431640625e-07


In [ ]:
plt.figure(figsize=(15,12))
for i in range(20):
    plt.plot([airports["LATITUDE"][i],airports["LATITUDE"][i+20]],[airports["LONGITUDE"][i],airports["LATITUDE"][i+20]])
plt.show()


In [ ]:
flights.loc[(flights["DESTINATION_AIRPORT"] =="ABR") & (flights["ORIGIN_AIRPORT"]=="ABR"),:].empty

In [ ]:
len([x for x in flights["DESTINATION_AIRPORT"].unique() if x in airports["IATA_CODE"].unique()])

In [ ]:
len(flights["DESTINATION_AIRPORT"].unique())